In [1]:
factor = 1.0
exactDate_mode = 'xor'

# Feature Matrix Generation

In chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb), Swissbib's goldstandard data has been processed to form records of pairs of duplicate and pairs of unique records. These records are the starting point for the final feature matrix generation and that is the reason, why the DataFrame was called feature base. As described in [[JudACaps](./A_References.ipynb#judacaps)], the next step will be an attribute-wise comparison of each attribute pair of each record in the original feature base. This comparison will generate similarity values for each attribute pair. It will halve the number of attributes replacing each attribute pair with one value expressing their degree of similarity. The goal of this chapter is a DataFrame with the full and final feature attributes. The values of these feature attributes will be used for training and performance testing of the machine learning models in the chapters to follow.

This chapter introduces similarity metrics for string comparisons. The metrics to be used for calculating its similarity will be decided for each attribute pair of the DataFrame built in the previous chapters.

## Table of Contents

- [Data Takeover](#Data-Takeover)
- [Object Distance and Similarity](#Object-Distance-and-Similarity)
    - [Mathematical Definitions](#Mathematical-Definitions)
    - [Library TextDistance](#Library-TextDistance)
- [Similarity Metrics on Attribute Level](#Similarity-Metrics-on-Attribute-Level)
    - [Table of Contents of Attribute Similarities](#Table-of-Contents-of-Attribute-Similarities)
- [DataFrame with Attributes and Similarity Features](#DataFrame-with-Attributes-and-Similarity-Features)
- [Summary](#Summary)
    - [Full Feature Matrix with Target Vector Handover](#Full-Feature-Matrix-with-Target-Vector-Handover)

## Data Takeover

Swissbib's raw data of the goldstandard has been processed in chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb). As the first step of this chapter, this data is loaded for further processing to the feature matrix and target vector for the subsequent machine learning model chapters.

In [2]:
import os
import pandas as pd
import pickle as pk
import bz2
import _pickle as cPickle

path_goldstandard = './daten_goldstandard'

# Restore metadata so far
with open(os.path.join(path_goldstandard, 'columns_metadata.pkl'), 'rb') as handle:
    columns_metadata_dict = pk.load(handle)

# Restore DataFrame with features from compressed pickle file
with bz2.BZ2File((os.path.join(
    path_goldstandard, 'feature_base_df.pkl')), 'rb') as file:
    df_feature_base = cPickle.load(file)

# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(df_feature_base.columns)

df_feature_base.head()

,035liste_x,035liste_y,century_x,century_y,coordinate_E_x,coordinate_E_y,coordinate_N_x,coordinate_N_y,coordinate_x,coordinate_y,corporate_110_x,corporate_110_y,corporate_710_x,corporate_710_y,corporate_full_x,corporate_full_y,decade_x,decade_y,docid_x,docid_y,doi_x,doi_y,duplicates,edition_x,edition_y,exactDate_x,exactDate_y,format_postfix_x,format_postfix_y,format_prefix_x,format_prefix_y,isbn_x,isbn_y,ismn_x,ismn_y,masters_docid,musicid_x,musicid_y,pages_x,pages_y,part_x,part_y,person_100_x,person_100_y,person_245c_x,person_245c_y,person_700_x,person_700_y,pubinit_x,pubinit_y,pubword_x,pubword_y,pubyear_x,pubyear_y,scale_x,scale_y,ttlfull_245_x,ttlfull_245_y,ttlfull_246_x,ttlfull_246_y,ttlpart_x,ttlpart_y,volumes_x,volumes_y
0,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (ABN)000539983]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,000311049,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem englischen übersetzt von...,"grawechristian, graweursula","grawechristian, graweursula",reclam jun.,reclam jun.,[Reclam jun.],[Reclam jun.],2009,2009,,,"emma, roman","emma, roman",,,"{'245': ['Emma', 'Roman']}","{'245': ['Emma', 'Roman']}",600,600
1,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (NEBIS)009587153]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,196506476,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem engl. übers. von ursula ...,"grawechristian, graweursula",,reclam jun.,reclam,[Reclam jun.],[Reclam],2009,2009,,,"emma, roman",emma,,,"{'245': ['Emma', 'Roman']}",{'245': ['Emma']},600,600
2,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (LIBIB)000315536]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,323173349,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen,"grawechristian, graweursula",,reclam jun.,reclam,[Reclam jun.],[Reclam],2009,2009,,,"emma, roman","emma, roman",,,"{'245': ['Emma', 'Roman']}","{'245': ['Emma', 'Roman']}",600,600
3,"[(OCoLC)731635279, (NEBIS)009587153]","[(OCoLC)731635279, (ABN)000539983]",2009,2009,,,,,[],[],,,,,,,2009,2009,196506476,000311049,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem englischen übersetzt von...,,"grawechristian, graweursula",reclam,reclam jun.,[Reclam],[Reclam jun.],2009,2009,,,emma,"emma, roman",,,{'245': ['Emma']},"{'245': ['Emma', 'Roman']}",600,600
4,"[(OCoLC)731635279, (NEBIS)009587153]","[(OCoLC)731635279, (NEBIS)009587153]",2009,2009,,,,,[],[],,,,,,,2009,2009,196506476,196506476,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem engl. übers. von ursula ...,,,reclam,reclam,[Reclam],[Reclam],2009,2009,,,emma,emma,,,{'245': ['Emma']},{'245': ['Emma']},600,600


Now, the feature base can be reduced to the columns used for processing.

In [3]:
# The DataFrame of pairs with target information
df_feature_base = df_feature_base[columns_metadata_dict['columns_to_use']]

df_feature_base.sample(n=5)

,duplicates,coordinate_E_x,coordinate_E_y,coordinate_N_x,coordinate_N_y,corporate_full_x,corporate_full_y,doi_x,doi_y,edition_x,edition_y,exactDate_x,exactDate_y,format_prefix_x,format_prefix_y,format_postfix_x,format_postfix_y,isbn_x,isbn_y,ismn_x,ismn_y,musicid_x,musicid_y,part_x,part_y,person_100_x,person_100_y,person_700_x,person_700_y,person_245c_x,person_245c_y,pubinit_x,pubinit_y,scale_x,scale_y,ttlfull_245_x,ttlfull_245_y,ttlfull_246_x,ttlfull_246_y,volumes_x,volumes_y
215273,0,,,,,,,,,,,2015uuuu,1994uuuu,bk,bk,020000,020000,[978-0-7294-0965-0],[],,,,,13,1,voltaire,,venturinodiego,,sous la dir. de diego venturino = [les oeuvres...,[sigrid kessler... et al.] ; [hrsg.] interkant...,voltaire foundation,[staatlicher lehrmittelverlag],,,"the complete works of voltaire, siècle de loui...","bonne chance!, cours de langue française, 1, d...",,,407,93
131874,0,,,,,,,,,,2,1982uuuu,20130627,bk,bk,020000,020053,[],"[978-3-648-04701-9, 3-648-04701-9 (EBook) : US...",,,,,3,,,,,"basuandreas, faustliane",sigrid kessler... [et al.] ; [éd.:] interkanto...,,staatlicher lehrmittelverl.,"r haufe verlag gmbh & co. kg, ebl - ebook libr...",,,"bonne chance!, cours de langue française, deux...","gewaltfreie kommunikation, taschenguide",,,256,1
63712,0,,,,,,,,,,,uuuuuuuu,2007uuuu,mu,bk,010200,020053,[],"[978-3-598-31808-5 (print), 978-3-11-092585-2]",,,5944,,,48 48,mozartwolfgang amadeus,mortzfeldpeter,schikanederemanuel,raabepaul,wolfgang amadeus mozart ; eine romantische ope...,"mortzfeld, peter; raabe, paul",,de gruyter saur,,,"die zauberflöte, il flauto magico : oper in zw...",katalog der graphischen porträts in der herzog...,"die zauberflöte, ausgabe für gesang und klavier",,1,1 374
197622,0,e0074147,,n0460833,,eidgenössisches topographisches bureau,,,,,,1862uuuu,1968uuuu,mp,bk,010300,020000,[],[],,,,,23 1862,,,mozartwolfgang amadeus,"dufourguillaume henri, müllhaupthans heinrich,...",schikanederemanuel,g.h. dufour direxit ; h. müllhaupt sculpsit,wolfgang amadeus mozart ; verf. von emmanuel s...,,,100000,,"domo d'ossola, arona","die zauberflöte, [textbuch] = the magic vlute ...",,,1,76
42823,0,,e0074147,,n0460833,,schweizeidgenössisches topographisches bureau,,,,,1993uuuu,1870uuuu,mu,mp,010100,010300,[963-8303-08-5],[],,,,,,23 23 1870 23,mozartwolfgang amadeus,,,"dufourguillaume-henri, müllhauptheinrich",wolfgang amadeus mozart,g. h. dufour direxit ; h. müllhaupt sculpsit,könemann,[eidg. topographisches bureau],,100000,"die zauberflöte, partitura","domo d'ossola, arona",,"[domodossola, arona]",225,1


In [4]:
print('Number of rows labelled as duplicates {:,d}'.format(len(df_feature_base[
    df_feature_base.duplicates==1])))
print('Number of rows labelled as uniques {:,d}'.format(len(df_feature_base[
    df_feature_base.duplicates==0])))
print('Total number of rows in DataFrame {:,d}'.format(df_feature_base.shape[0],
      'number of columns', df_feature_base.shape[1]))

Number of rows labelled as duplicates 1,473
Number of rows labelled as uniques 257,955
Total number of rows in DataFrame 259,428


In [5]:
print('Part of duplicates (1) and uniques (0) in units of [%]')
print(round(100*df_feature_base.duplicates.value_counts(normalize=True), 2))

Part of duplicates (1) and uniques (0) in units of [%]
0    99.43
1     0.57
Name: duplicates, dtype: float64


DataFrame feature base is the starting point used for the further processing in this chapter.

## Object Distance and Similarity

A mathematical idea of distance and similarity is needed for understanding object pair comparison. This section starts with a motivation for calculating similarities and afterwards gives a very basic definition of the two central terms, distance and similarity. The text of this section is a summary of [[Chri2012](./A_References.ipynb#chri2012)].

### Mathematical Definitions

The attributes to be used for pair comparison may contain values of poor quality. The quality originates in the way the data has been entered at the very source. Manual data entry may suffer from mistyping, automatically scanned data may suffer from insufficiencies of the scanned base material or the recognition algorithm in the optical character recognition (OCR) processing. The basic step of a deduplication process is to identify the probability of two strings of a pair to be a pair of duplicates. This is done by calculating a similarity value between the two strings compared, rather than using an exact comparison function. Based on this common similarity value for an attribute pair, their being duplicates can be decided.

The term similarity is strongly coupled to the term of distance of two values of an attribute. Mathematically, a distance can be explained with the help of a distance function. A _distance function_ or _distance metric_ $dist(o_i, o_j)$ between two points or data objects $o_i$ and $o_j$ must fulfill four requirements.

1. $dist(o_i, o_i)=0$, the distance from an object to itself is zero.
- $dist(o_i, o_j)\ge 0$, the distance between two objects is a non-negative number.
- $dist(o_i, o_j)=dist(o_j, o_i)$, the distance between two objects is symmetric.
- $dist(o_i, o_j)\le dist(o_i, o_k)+dist(o_k, o_j)$, the triangular inequality must hold. It states that the direct distance beween two objects is never larger than the combined distance when going through a third object.

A distance value expresses the dissimilarity $d$ of two objects [[HanK2012](./A_References.ipynb#hank2012)] and can therefore be converted into a similarity value $s$, calculating $s = \frac{1}{d}$, assuming $d\gt 0$. Alternatively, assuming the distance value is normalised $0\le d\le 1$, the similarity value can be calculated to $s = 1-d$. A _similarity function_ $sim(a_i, aj)$ between two attributes which can be strings, numbers, dates, geographic locations, text, XML documents, etc. fulfills the general requirements.

1. $sim(a_i, a_i)=1$, the result of comparing a value with itself is an exact similarity.
- $sim(a_i, a_j)=0$, the similarity of values that are completely different from each other is 0. What accounts for 'complete different' depends upon the type of data that are compared.
- $0\lt sim(a_i, a_j)\lt 1$, an approximate similarity between exact similarity and total dissimilarity is calculated if two attribute values are somewhat similar to each other. What accounts for 'somewhat different' depends upon the type of data that are compared.

The dissimilarity between two objects $o_i$ and $o_j$ can be computed based on the ratio of mismatches,
$$
d(o_i, o_j) = \frac{p-m}{p},
$$
where $m$ is the number of matching attributes and $p$ is the total number of attributes describing the objects [[HanK2012](./A_References.ipynb#hank2012)]. Thus the similarity between two objects can be computed as
$$
sim(o_i, o_j) = 1 - d(o_i, o_j) = \frac{m}{p}.
$$

For data deduplication, a comparison function needs to be tailored to the type of underlying data. Although there is a correspondence between a similarity function and the mathematical concept of a distance function, not all known and implemented similarity comparison functions used for string pair comparison fulfill the requirements of a distance function. Some similarity functions are not symmetric, others do not fulfill the triangular inequality. Decision taking on the best similarity function for a string pair, will be based on the effect, a similarity function has for the purpose needed. In the case of this capstone project, this purpose is its capability to contribute to the prediction whether a pair of records is a pair duplicates or a pair of uniques.

### Library TextDistance

An internet research on string distance calculation with Python has revealed libraries [[StSi](./A_References.ipynb#stsi)], [[TeDi](./A_References.ipynb#tedi)] and seperate code snippets for distinct algorithms. After trying the referenced libraries and a downloaded code snippet for a Smith Waterman similarity [[SmWa](./A_References.ipynb#smwa)], the text distance library [[TeDi](./A_References.ipynb#tedi)] has been decided to be the best decision for this capstone project. The decision is based on the github statistics of stars and the date of the latest pull requests, indicating its popularity and maintenance activity of the library. A look at the API of the library, reveals the Python library to be a complete implementation (compared to suggestions of similarity metrics in [[Chri2012](./A_References.ipynb#chri2012)]) and easy to use.

In [6]:
# Install textdistance Python library - if not done, yet.
! pip install textdistance

For using the library, see documentation in [[TeDi](./A_References.ipynb#tedi)]. For the purposes of this chapter, function $\texttt{.normalized}\_\texttt{similarity}()$ of an instantiated textdistance object will be used.

In [7]:
import textdistance as tedi

With the code line above, the library is imported for application in this chapter. In appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb) the effects of the similarity metrics of the library are compared for better understanding of their specific behaviour. This comparison for each attribute is the basis of deciding the best similarity metric available for an attribute pair.

## Similarity Metrics on Attribute Level

This section implements the decision for calculating the similarity metric for each attribute of the raw data based on appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb). The implementation is applied on a pair of attributes of two records, resulting in a new attribute, the similarity value, of the final feature matrix. A general function $\texttt{.build}\_\texttt{delta}\_\texttt{feature}()$ is provided by the code file [data_preparation_funcs.py](./data_preparation_funcs.py) for transforming two attributes into their feature attribute holding their similarity value.

In [8]:
import data_preparation_funcs as dpf

The two dictionaries of the following code cell will be filled by function $\texttt{.build}\_\texttt{delta}\_\texttt{feature}()$.

In [9]:
columns_metadata_dict['similarity_metrics'] = {}
columns_metadata_dict['features'] = []

### Table of Contents of Attribute Similarities

- [coordinate](#coordinate)
- [corporate](#corporate)
- [doi](#doi)
- [edition](#edition)
- [exactDate](#exactDate)
- [format](#format)
- [isbn](#isbn)
- [ismn](#ismn)
- [musicid](#musicid)
- [part](#part)
- [person](#person)
- [pubinit](#pubinit)
- [scale](#scale)
- [ttlfull](#ttlfull)
- [volumes](#volumes)

### coordinate

As discussed in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{coordinate}$ holds coordinates of maps. To decide whether a map covers the same geographical range, a metric will be chosen that compares the coordinate number digits from left to right. The more digits are found to be equal, the higher the similarity value is calculated. The comparison stops with the first digit pair that differs. This algorithm is satisfied by the LCS (Longest Common Substring comparison) algorithm and generates the wanted result, see appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb).

In [10]:
attribute = 'coordinate'

columns_metadata_dict['similarity_metrics'][attribute+'_E'] = tedi.LCSStr()
columns_metadata_dict['similarity_metrics'][attribute+'_N'] = tedi.LCSStr()

ne_values = ['_E', '_N']

for ne in ne_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+ne,
        columns_metadata_dict['similarity_metrics'][attribute+ne],
        columns_metadata_dict)

The length of attribute $\texttt{coordinate}$ is exactly eight digits. The distinct similarity values that may occur form a discrete set of values with a distance of $\frac{1}{8}$ between adjacent values.

In [11]:
uniques, uniques_len = {}, {}

for ne in ne_values :
    uniques[attribute+ne], uniques_len[attribute+ne] = dpf.determine_similarity_values(
        df_feature_base, attribute+ne)

coordinate_E values range [0.    0.375 0.5   0.625 0.875 1.   ]
coordinate_N values range [0.    0.375 0.5   0.75  0.875 1.   ]


Looking at some samples of the feature matrix reveals a good match to the expectations.

In [12]:
position = 3

for ne in ne_values :
    dpf.show_samples_interval(
        df_feature_base, attribute+ne,
        uniques[attribute+ne][uniques_len[attribute+ne]-position],
        uniques[attribute+ne][uniques_len[attribute+ne]-position+1]
    )

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y
869,1,0.875,e0080855,e0080851
196658,0,0.625,e0055700,e0055009
96189,0,0.875,e0080851,e0080855
865,1,0.875,e0080851,e0080855
854,1,0.875,e0080855,e0080851


0.625 <= coordinate_E_delta <= 0.875


,duplicates,coordinate_N_delta,coordinate_N_x,coordinate_N_y
69495,0,0.75,n0460826,n0460833
182748,0,0.75,n0460826,n0460833
184535,0,0.75,n0460833,n0460826
182761,0,0.75,n0460826,n0460833
182770,0,0.75,n0460826,n0460833


0.75 <= coordinate_N_delta <= 0.875


The samples above show the wanted similarity behaviour for value ranges greater than 0. The metric has the weakness, though, that empty coordinate values, e.g. for bibliographic units other than maps, have each been calculated to a similarity of 0. Some samples for duplicates in the training data are shown below.

In [13]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1],
    attribute+'_E', uniques[attribute+'_E'][0], uniques[attribute+'_E'][1], 10)

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y
747,1,0.0,,
348,1,0.0,,
199,1,0.0,,
684,1,0.0,,
702,1,0.0,,
510,1,0.0,,
92,1,0.0,,
449,1,0.0,,
892,1,0.0,,
1300,1,0.0,,


0.0 <= coordinate_E_delta <= 0.375


This downside shall be avoided by marking pairs of missing coordinate values on both sides with a special negative value, which will point out to the models to be trained, the special case of missing information in a row. The implementation of this logic is done in function $\texttt{.mark}\_\texttt{missing}()$. The absolute value of this negative number is conrolled by a factor which is passed to the function as a parameter. The function handles explicitly two cases. The first one is missing information in both attributes (resulting in $-1*\texttt{factor}$) and the second one is missing information in only one of the two attributes (resulting in $-0.5*\texttt{factor}$).

In [14]:
for ne in ne_values :
    df_feature_base = dpf.mark_missing(df_feature_base, attribute+ne, factor)

### corporate

Attribute $\texttt{corporate}$ is a collection of corporate names. The Monge-Elkan metric compares string tokens pairwise [[Chri2012](./A_References.ipynb#chri2012)] while the LCS metric searches for the longest common substring. Assessing the differences of these two metrics with the help of their values distribution in chapter [Features Discussion and Dummy Classifier Baseline](./5_FeatureDiscussionDummyBaseline.ipynb), reveals a better distribution behaviour for LCS. Therefore, the LCS metric will be chosen for this attribute.

In [15]:
attribute = 'corporate_full'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.LCSStr()
#tedi.StrCmp95()
#tedi.MongeElkan()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [16]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

corporate_full values range [0.         0.01333333 0.01428571 0.01639344 0.01666667 0.01694915
 0.01818182 0.01960784 0.02       0.02040816 0.0212766  0.02173913
 0.02272727 0.02380952 0.025      0.02531646 0.02542373 0.02666667
 0.02702703 0.02727273 0.02857143 0.03       0.03030303 0.03125
 0.03225806 0.03278689 0.03333333 0.03389831 0.03448276 0.03571429
 0.03636364 0.03773585 0.03797468 0.03846154 0.03921569 0.04
 0.04081633 0.04166667 0.04237288 0.04255319 0.04285714 0.04347826
 0.04444444 0.04545455 0.046875   0.04761905 0.04878049 0.04918033
 0.05       0.05263158 0.05333333 0.05405405 0.05454545 0.05555556
 0.05660377 0.05714286 0.05882353 0.05932203 0.06       0.06060606
 0.06122449 0.0625     0.06329114 0.06363636 0.06382979 0.06451613
 0.06521739 0.06557377 0.06666667 0.06779661 0.06818182 0.06896552
 0.07       0.07142857 0.07272727 0.07317073 0.075      0.0754717
 0.07594937 0.07627119 0.07692308 0.078125   0.07843137 0.07894737
 0.08       0.08108108 0.08163265 0.08181818

Its $110$ part is sparsely filled and even its $710$ part comes along with a little more than $10\%$ of filling, only. The LCS metric generates a similarity of 1 for the cases where both strings of a pair are empty. Missing values on both sides may be an indicator for a pair of duplicates but due to the sparsely available information, it is a weak indicator. Therefore, the pairs with missing data on both sides of the pair, will be marked with the negative value.

In [17]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

Some sample cases are shown below for both $\texttt{corporate}$ features.

In [18]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1],
    attribute, 0.0, 1.0, 20
)

,duplicates,corporate_full_delta,corporate_full_x,corporate_full_y
865,1,1.000000,eidgenössische landestopographie,eidgenössische landestopographie
17,1,0.527273,"metropolitan operaorchestra, metropolitan oper...","metropolitan opera orchestra, metropolitan ope..."
204,1,0.490000,schweizerische gesellschaft für bildungsforschung,schweizerische gesellschaft für bildungsforsch...
1155,1,1.000000,"interkantonale lehrmittelzentrale (rapperswil,...","interkantonale lehrmittelzentrale (rapperswil,..."
1417,1,1.000000,schweizerische normen-vereinigung,schweizerische normen-vereinigung
1156,1,1.000000,"philharmonia chorus (london), philharmonia orc...","philharmonia chorus (london), philharmonia orc..."
12,1,0.400000,"metropolitan opera orchestra, metropolitan ope...","metropolitan opera (new york)orchestra, metrop..."
1312,1,0.844444,eidgenössisches topographisches bureau,schweizeidgenössisches topographisches bureau
1382,1,1.000000,schweizerische normen-vereinigung,schweizerische normen-vereinigung
206,1,1.000000,schweizerische gesellschaft für bildungsforschung,schweizerische gesellschaft für bildungsforschung


0.0 <= corporate_full_delta <= 1.0


In [19]:
position = uniques_len[attribute]//2 # Let's have a look in the middle range of the similarities.

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 20)

,duplicates,corporate_full_delta,corporate_full_x,corporate_full_y
168196,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau
47859,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenössisches topographisches bureau
86955,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau
47817,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau
12238,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau
12279,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenössisches topographisches bureau
86954,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau
25878,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau
38371,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenössisches topographisches bureau
47819,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau


0.10126582278481011 <= corporate_full_delta <= 0.10204081632653061


### doi

Swissbib uses an explicit $\texttt{doi}$ attribute for its deduplication implementation. In chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb), the real doi identifier has been isolated with the help of a preprocessing function $\texttt{.reduce}\_\texttt{to}\_\texttt{doi}\_\texttt{element}()$, see [Data Analysis](./1_DataAnalysis.ipynb). Attribute $\texttt{doi}$ contains a single string value. The Identity metric will be used for comparing the string values of a pair in a row, calculating a similarity value of 1.0 or 0.0 for each pair. If one list is empty a value of 0 is returned.

In [20]:
attribute = 'doi'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

df_feature_base['doi_delta'].unique()

array([1., 0.])

Some sample cases are shown below for each category of $\texttt{doi}\_\texttt{delta}$.

In [21]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

for doi_delta_value in df_feature_base['doi_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['doi_delta']==doi_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'doi', doi_delta_value, number_of_max_samples)
    print(f'doi_delta = {doi_delta_value}')

doi values range [0. 1.]


,duplicates,doi_delta,doi_x,doi_y
25841,0,1.0,,
105227,0,1.0,,
161457,0,1.0,,
119173,0,1.0,,
213164,0,1.0,,
61185,0,1.0,,
201775,0,1.0,,
93043,0,1.0,,
39134,0,1.0,,
65356,0,1.0,,


doi_delta = 1.0


,duplicates,doi_delta,doi_x,doi_y
253399,0,0.0,10.1055/b-005-143650,
4896,0,0.0,,10.5169/seals-583145
190476,0,0.0,10.1055/b-002-26639,
166609,0,0.0,10.1093/ndt/gft319,
208538,0,0.0,10.1093/ndt/gft319,
177950,0,0.0,,10.5169/seals-376510
110372,0,0.0,,10.5169/seals-377362
39202,0,0.0,,10.5169/seals-377028
177501,0,0.0,,10.1007/978-3-642-41698-9
190094,0,0.0,10.1055/b-002-26639,


doi_delta = 0.0


In [22]:
# Let's have a look at some non-empty doi elements
df_doi_with_element = df_feature_base[df_feature_base.doi_x.apply(lambda x : len(x) > 0)]

for doi_delta_value in df_feature_base['doi_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['doi_delta']==doi_delta_value])
    )

    dpf.show_samples_distinct(df_doi_with_element, 'doi', doi_delta_value, number_of_max_samples)
    print(f'doi_delta = {doi_delta_value}')

,duplicates,doi_delta,doi_x,doi_y
207982,0,1.0,10.1093/cid/cir669,10.1093/cid/cir669
1261,1,1.0,10.1093/ndt/gft319,10.1093/ndt/gft319
1471,1,1.0,10.1055/b-005-143650,10.1055/b-005-143650
1255,1,1.0,10.1007/978-3-642-41698-9,10.1007/978-3-642-41698-9
1265,1,1.0,10.1093/cid/ciu795,10.1093/cid/ciu795
200628,0,1.0,10.1007/978-3-642-41698-9,10.1007/978-3-642-41698-9
1268,1,1.0,10.1093/cid/ciu795,10.1093/cid/ciu795
1472,1,1.0,10.1055/b-005-143650,10.1055/b-005-143650
167420,0,1.0,10.1007/978-3-642-41698-9,10.1007/978-3-642-41698-9
1469,1,1.0,10.1055/b-005-143650,10.1055/b-005-143650


doi_delta = 1.0


,duplicates,doi_delta,doi_x,doi_y
154506,0,0.0,10.5167/uzh-53042,
167802,0,0.0,10.1093/cid/ciu795,
166666,0,0.0,10.1093/ndt/gft319,
167598,0,0.0,10.1093/cid/ciu795,
255765,0,0.0,10.1055/b-005-143650,
167059,0,0.0,10.1007/978-3-642-41698-9,
154702,0,0.0,10.5167/uzh-53042,
252954,0,0.0,10.1055/b-005-143650,
200360,0,0.0,10.1007/978-3-642-41698-9,
200187,0,0.0,10.1007/978-3-642-41698-9,


doi_delta = 0.0


As can be seen above, a value of 1.0 is returned if both strings of a pair are empty. As the attribute filling of $\texttt{doi}$ is sparse, see chapter [Data Analysis](./1_DataAnalysis.ipynb), the $\texttt{doi}\_\texttt{delta}$ indicates strongly a pair of duplicates for most rows. To avoid such misleading identity indication, function $\texttt{.mark}\_\texttt{missing}()$ will be applied to the attribute.

In [23]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### edition

In its original form in Swissbib's raw data, the edition statement is a string value which may have several words. The modelling on this attribute has been tried with and without stripping letter characters from the string. The final decision for the best processing will be documented in chapter [Overview and Summary](./0_OverviewSummary.ipynb). A Jaccard similarity is tried for this attribute.

In [24]:
attribute = 'edition'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [25]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

import numpy as np

edition_delta_uniques = np.sort(df_feature_base['edition_delta'].unique())
edition_delta_uniques_len = len(edition_delta_uniques)
print('edition values range', edition_delta_uniques[:30])

edition values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.4        0.5        0.6        1.        ]
edition values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.4        0.5        0.6        1.        ]


The comparison results in a wide number of distinct similarity values for the goldstandard data set. Below, some examples are shown.

In [26]:
position = edition_delta_uniques_len

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position],
    edition_delta_uniques[edition_delta_uniques_len-position+2], 10)

position = edition_delta_uniques_len//2

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)

,duplicates,edition_delta,edition_x,edition_y
229999,0,1.0,,
6394,0,1.0,,
69581,0,1.0,,
244739,0,1.0,,
62938,0,1.0,,
92975,0,1.0,,
91886,0,1.0,,
40868,0,1.0,,
143236,0,1.0,,
65244,0,1.0,,


0.6 <= edition_delta <= 1.0


,duplicates,edition_delta,edition_x,edition_y
154477,0,0.0,,13
180881,0,0.0,6,
53077,0,0.0,,87
237312,0,0.0,,2000
123973,0,0.0,,11
49746,0,0.0,,2
199756,0,0.0,2,
57942,0,0.0,19,
188362,0,0.0,,1905
174288,0,0.0,6,


0.0 <= edition_delta <= 0.1428571428571428


,duplicates,edition_delta,edition_x,edition_y
192294,0,0.25,2,2007
194039,0,0.25,2,1928
97459,0,0.25,1791,7
28845,0,0.25,5,1905
42066,0,0.25,1791,1
123136,0,0.25,2,1926
42310,0,0.25,1791,7
54940,0,0.25,1,1909
199633,0,0.25,2,1926
145401,0,0.25,3,1943


0.19999999999999996 <= edition_delta <= 0.25


Again, for $\texttt{edition}\_\texttt{delta} = 1$, many empty values of the $\texttt{edition}$ attribute can be observed. These will be marked with the special negative value in the data with the goal to distinguish them from the matching attribute pairs.

In [27]:
df_feature_base = dpf.mark_missing(df_feature_base, 'edition', factor)

In [28]:
position = edition_delta_uniques_len

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)

,duplicates,edition_delta,edition_x,edition_y
675,1,1.0,5,5
1195,1,1.0,6,6
669,1,1.0,5,5
68737,0,1.0,2,2
184790,0,1.0,6,6
192081,0,1.0,2,2
1353,1,1.0,2,2
109629,0,1.0,4,4
215937,0,1.0,6,6
180639,0,1.0,6,6


0.6 <= edition_delta <= 1.0


### exactDate

As discussed in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{exactDate}$ holds a year number stored in the first four digits. Letter 'u' is used as a placeholder for an unknown digit. The attribute may hold some month and day or a second year information in its second four digits, additionally.

The attribute will be kept as a string and will not be transformed to an integer. The feature attribute of the record pair to be compared will be calculated with a modified Hamming algorithm, see appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb). The resulting similarity will be stored in a new attribute $\texttt{exactDate}\_\texttt{delta}$ which will be taken for the model calculation.

As can be seen in chapter [Decision Tree Model](./6_DecisionTreeModel.ipynb), this attribute is important for prediction. Different ways of increasing the weight of the unknown status of a digit have been tried. The different ways can be seen in the implementations below. The algorithm decided for the final simulation will be documented in chapter [Overview and Summary](./0_OverviewSummary.ipynb).

In [29]:
import string

def no_xor (x_side, y_side) :
    number = 0
    for i in range(len(x_side)) :
        if ((x_side[i] in string.ascii_lowercase) | (y_side[i] in string.ascii_lowercase)) & (x_side[i] != y_side[i]) :
            number = number + 1
    return number

print('Example comparison results in a value of', no_xor ('202a0aaa', '1920uuuu'))

Example comparison results in a value of 5


In [30]:
attribute = 'exactDate'

# Replace letter 'u' with letter 'a' for one of the two strings.
#  As an effect, the resulting Hamming similarity for a letter
#  instead of a numerical digit in either string will add with an amount 0.
df_feature_base[attribute+'_x'] = df_feature_base.exactDate_x.str.replace('u', 'a')

# Compute Hamming similarity for century string pair.
columns_metadata_dict['similarity_metrics'][attribute] = tedi.Hamming()

unknown_share = 16

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

if exactDate_mode == 'added_u':
    # Add amount of 1/16 to Hamming similarity for every letter digit.
    #  But only maximum number of letter digits in both strings of a pair.
    df_feature_base[attribute+'_delta'] = df_feature_base[[
        attribute+'_x', attribute+'_y', attribute+'_delta']].apply(
        lambda x : x[attribute+'_delta'] + 
        max(x[attribute+'_x'].count('a'), x[attribute+'_y'].count('u'))/unknown_share, axis=1
    )
elif exactDate_mode == 'xor':
    # Add amount of 1/16 to Hamming similarity for every letter digit.
    #  But only number of position-wise xor-ed letter digits in the two strings of a pair.
    df_feature_base[attribute+'_delta'] = df_feature_base[[
        attribute+'_x', attribute+'_y', attribute+'_delta']].apply(
        lambda x : x[attribute+'_delta'] + 
        no_xor(x[attribute+'_x'], x[attribute+'_y'])/unknown_share, axis=1
    )

In [31]:
df_feature_base[['exactDate_x', 'exactDate_y', 'exactDate_delta']].sample(n=10)

,exactDate_x,exactDate_y,exactDate_delta
62530,18aaaaaa,2008uuuu,0.375
27061,1998aaaa,1863uuuu,0.375
42386,19561791,2017uuuu,0.250
151109,2012aaaa,19942008,0.250
189944,2016aaaa,2012uuuu,0.625
213236,2013aaaa,1953uuuu,0.375
120109,1987aaaa,1978uuuu,0.500
193555,2016aaaa,1988uuuu,0.250
126754,18901897,19679999,0.250
158470,20092005,2014uuuu,0.500


All resulting values of equal strings are equal to 1.

In [32]:
df_feature_base[['exactDate_x', 'exactDate_y', 'exactDate_delta']][
    df_feature_base.exactDate_x == df_feature_base.exactDate_y
].sort_values('exactDate_delta', ascending=False).head()

,exactDate_x,exactDate_y,exactDate_delta
159,20022000,20022000,1.0
843,19849999,19849999,1.0
845,19969999,19969999,1.0
846,19969999,19969999,1.0
847,19969999,19969999,1.0


A discrete set of different similarity values can be found in the attribute deltas. Some sample records are shown below.

In [33]:
exactDate_deltas = np.sort(df_feature_base.exactDate_delta.unique())
exactDate_deltas

array([0.    , 0.125 , 0.25  , 0.3125, 0.375 , 0.4375, 0.5   , 0.5625,
       0.625 , 0.6875, 0.75  , 0.875 , 1.    ])

In [34]:
sample_size = 5

for i in exactDate_deltas :
    dpf.show_samples_distinct(df_feature_base, 'exactDate', i, sample_size)
    print(f'exactDate_delta = {i}')

,duplicates,exactDate_delta,exactDate_x,exactDate_y
184909,0,0.0,20159999,19972000
6418,0,0.0,19791999,20022003
121845,0,0.0,19911794,20162007
26320,0,0.0,19791999,20130627
124113,0,0.0,19829999,20002005


exactDate_delta = 0.0


,duplicates,exactDate_delta,exactDate_x,exactDate_y
208929,0,0.125,201310aa,19669999
43708,0,0.125,20091990,18631869
126458,0,0.125,19959999,15501850
154005,0,0.125,18501875,19669999
6718,0,0.125,19791999,20151475


exactDate_delta = 0.125


,duplicates,exactDate_delta,exactDate_x,exactDate_y
140863,0,0.25,1960aaaa,2007uuuu
233522,0,0.25,2014aaaa,1991uuuu
176768,0,0.25,2015aaaa,1988uuuu
229687,0,0.25,1959aaaa,2014uuuu
191404,0,0.25,2016aaaa,1489uuuu


...

exactDate_delta = 0.75


,duplicates,exactDate_delta,exactDate_x,exactDate_y
51434,0,0.875,19969999,19989999
51322,0,0.875,19969999,19989999
126040,0,0.875,19959999,19989999
34263,0,0.875,19959999,19969999
89703,0,0.875,19969999,19669999


exactDate_delta = 0.875


,duplicates,exactDate_delta,exactDate_x,exactDate_y
289,1,1.0,20071990,20071990
210,1,1.0,19791999,19791999
772,1,1.0,19561791,19561791
222,1,1.0,19791999,19791999
1285,1,1.0,20159999,20159999


exactDate_delta = 1.0


### format

Due to the discussion in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{format}$ has been split up into two new attributes $\texttt{format}\_\texttt{prefix}$ and $\texttt{format}\_\texttt{postfix}$ which will be compared by a different similarity metrics.

- As the quality of $\texttt{format}\_\texttt{prefix}$ is expected to be high, an identity comparison should be sufficient.
- Due to the observed structure of $\texttt{format}\_\texttt{postfix}$, a q-gram based comparison will be chosen.

In [35]:
attribute = 'format'

columns_metadata_dict['similarity_metrics'][attribute+'_prefix'] = tedi.Identity()
columns_metadata_dict['similarity_metrics'][attribute+'_postfix'] = tedi.Jaccard(qval=2)

pfix_values = ['_prefix', '_postfix']

for pf in pfix_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+pf,
        columns_metadata_dict['similarity_metrics'][attribute+pf],
        columns_metadata_dict)

In [36]:
for i in df_feature_base.format_prefix_delta[
    df_feature_base.format_prefix_x != df_feature_base.format_prefix_y].unique():
    
    dpf.show_samples_distinct(df_feature_base, 'format_prefix', i)
    print(f'format_prefix_delta = {i}')

,duplicates,format_prefix_delta,format_prefix_x,format_prefix_y
109141,0,0.0,bk,mu
46640,0,0.0,vm,bk
134704,0,0.0,mu,vm
190482,0,0.0,bk,mp
102411,0,0.0,bk,mu


format_prefix_delta = 0.0


In [37]:
for i in df_feature_base.format_postfix_delta[
    df_feature_base.format_postfix_x != df_feature_base.format_postfix_y].unique():
    
    dpf.show_samples_distinct(df_feature_base, 'format_postfix', i)
    print(f'format_postfix_delta = {i}')

,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
240984,0,0.428571,020000,020053
233636,0,0.428571,020053,020000
76301,0,0.428571,010200,010300
96238,0,0.428571,010300,030000
82296,0,0.428571,040100,030100


format_postfix_delta = 0.4285714285714286


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
6774,0,0.111111,030600,020053
135924,0,0.111111,020000,010300
119624,0,0.111111,020000,030100
47901,0,0.111111,030600,020000
69723,0,0.111111,020053,040100


format_postfix_delta = 0.11111111111111116


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
101551,0,0.25,010200,020353
247213,0,0.25,030653,030000
108878,0,0.25,020000,020353
43482,0,0.25,020300,030653
124296,0,0.25,020000,020353


...

format_postfix_delta = 0.0


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
138570,0,1.0,020000,020000
226849,0,1.0,020000,020000
39074,0,1.0,020000,020000
45239,0,1.0,020000,020000
70231,0,1.0,020053,020053


format_postfix_delta = 1.0


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
245199,0,0.666667,020400,020047
88975,0,0.666667,030500,030053


format_postfix_delta = 0.6666666666666666


### isbn

Swissbib uses each string element of the $\texttt{isbn}$ list separately for comparing with each string element of its comparison $\texttt{isbn}$ list. If two bibliographic units hold at least one element in common, this is interpreted as a strong indicator for duplicates [[WiCo2001](./A_References.ipynb#wico2001)].

This hard logic is used in a modified way in the context of this capstone project. A special comparison function $\texttt{.build}\_\texttt{delta}\_\texttt{isbn}()$ has been implemented that compares each list element of the left-hand side with each list element of the right-hand side of a pair. According to Swissbib's implementation, the Identity metric is used for string comparison, calculating a similarity value of 1.0 or 0.0 for each list element pair. For normalisation reasons, the sum of similarity values is divided by the number of elements of the smaller list. If both lists are empty a value of 1.0 is returned. If only one list is empty a value of 0.0 is returned.

In [38]:
attribute = 'isbn'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

df_feature_base[attribute+'_delta'].unique()

array([1. , 0. , 0.5])

Some sample cases are shown below for each category of $\texttt{isbn_delta}$.

In [39]:
for isbn_delta_value in df_feature_base['isbn_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['isbn_delta']==isbn_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'isbn', isbn_delta_value, number_of_max_samples)
    print(f'isbn_delta = {isbn_delta_value}')

,duplicates,isbn_delta,isbn_x,isbn_y
160411,0,1.0,[],[]
7518,0,1.0,[],[]
60635,0,1.0,[],[]
78215,0,1.0,[],[]
127943,0,1.0,[],[]
150181,0,1.0,[],[]
92453,0,1.0,[],[]
58462,0,1.0,[],[]
18291,0,1.0,[],[]
150204,0,1.0,[],[]


isbn_delta = 1.0


,duplicates,isbn_delta,isbn_x,isbn_y
159404,0,0.0,[978-3-643-12370-1],[]
21187,0,0.0,[3-499-17476-6],[]
27414,0,0.0,[3-932992-42-3],[]
189241,0,0.0,[978-3-648-07838-9],[]
195425,0,0.0,"[978-3-13-127286-7, 3-13-127286-4]",[0252-9955]
183891,0,0.0,[0-87834-101-3],[]
182034,0,0.0,[978-3-7190-3171-8],"[978-3-292-00266-2 (Livre du professeur, Cahie..."
28919,0,0.0,[3-254-08008-4],[]
91094,0,0.0,[3-932992-42-3],[978-0-14-062010-8]
209608,0,0.0,[978-3-13-127286-7],"[978-3-598-31508-4 (print), 978-3-11-096992-4]"


isbn_delta = 0.0


,duplicates,isbn_delta,isbn_x,isbn_y
1195,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"
1202,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"
1210,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"
1199,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"
1201,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"
1205,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"


isbn_delta = 0.5


For attribute $\texttt{isbn}$, the special marking of missing values is omitted.

### ismn

This attribute will be processed with the identity similarity metric. The reasoning for this decision is the same as for similar attributes above. 

In [40]:
attribute = 'ismn'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()
#tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [41]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

ismn values range [0. 1.]


In [42]:
for ismn_delta_value in df_feature_base[attribute+'_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base[attribute+'_delta']==ismn_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'ismn', ismn_delta_value, number_of_max_samples)
    print(f'ismn_delta = {ismn_delta_value}')

,duplicates,ismn_delta,ismn_x,ismn_y
155397,0,1.0,,
147550,0,1.0,,
229250,0,1.0,,
46785,0,1.0,,
94265,0,1.0,,
200641,0,1.0,,
59311,0,1.0,,
127510,0,1.0,,
86972,0,1.0,,
123239,0,1.0,,


ismn_delta = 1.0


,duplicates,ismn_delta,ismn_x,ismn_y
98964,0,0.0,m006546749,
249458,0,0.0,9790006450510,
135334,0,0.0,m006450510,
20051,0,0.0,m006450510,
91511,0,0.0,,9790006201334
115187,0,0.0,"m006546756 (kritischer bericht, leinen)",
135054,0,0.0,m006450510,
222409,0,0.0,,m008060205
102946,0,0.0,m006450510,
157927,0,0.0,,9790006201334


ismn_delta = 0.0


As can be seen in the previous chapters, attribute $\texttt{ismn}$ is filled sparsely. A lot of missing values calculate to a value of 1.0 in the chosen similarity metrics. To mark these cases specifically, they will be transformed to a negative value.

In [43]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### musicid

Chapter [Data Analysis](./1_DataAnalysis.ipynb) shows that attribute $\texttt{musicid}$ is an identifyer for a music record. A Jaccard metric has been tested on this attribute, resulting in a distribution of many high similarity values on uniques. Comparing this result with the LCS metric, the latter has been decided.

In [44]:
attribute = 'musicid'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.LCSStr()
#tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [45]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

musicid values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.375      0.4        0.42857143 0.5
 0.6        0.66666667 0.75       1.        ]


In [46]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

position = uniques_len[attribute]//2

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)

,duplicates,musicid_delta,musicid_x,musicid_y
164237,0,1.0,4553,4553
1023,1,1.0,0630,0630
1021,1,1.0,0630,0630
789,1,1.0,502430,502430
36629,0,1.0,422,422
489,1,1.0,10425,10425
472,1,1.0,245,245
281,1,1.0,502023,502023
37395,0,1.0,4553,4553
116726,0,1.0,4553,4553


0.75 <= musicid_delta <= 1.0


,duplicates,musicid_delta,musicid_x,musicid_y
202571,0,0.0,,
255825,0,0.0,,
102848,0,0.0,,
162728,0,0.0,,
243826,0,0.0,,
90632,0,0.0,,
80417,0,0.0,,
125041,0,0.0,,
112048,0,0.0,,
230242,0,0.0,10425,


0.0 <= musicid_delta <= 0.1428571428571429


,duplicates,musicid_delta,musicid_x,musicid_y
64504,0,0.4,422,10425
88074,0,0.4,422,10425
74335,0,0.4,10425,426
134467,0,0.4,10425,1092
18593,0,0.4,10425,426
148827,0,0.4,433,33001
15875,0,0.4,422,10425
169630,0,0.4,0630,99064
141093,0,0.4,10425,422
138036,0,0.4,242,10425


0.375 <= musicid_delta <= 0.4


In [47]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1], attribute,
    uniques[attribute][0],
    uniques[attribute][uniques_len[attribute]-1], 20)

,duplicates,musicid_delta,musicid_x,musicid_y
737,1,1.0,4553,4553
1067,1,0.0,,
102,1,0.0,,
205,1,0.0,,
346,1,0.0,,
1095,1,0.0,,
1101,1,0.0,,
67,1,0.0,,
890,1,0.0,,
685,1,0.0,,


0.0 <= musicid_delta <= 1.0


The attribute is filled with a degree of below $10\%$. The chosen metric for it results in a similarity value of 1.0 for empty value pairs. This effect can be adjusted with function $\texttt{.mark}\_\texttt{missing}()$ as above. 

In [48]:
df_feature_base = dpf.mark_missing(df_feature_base, 'musicid', factor)

### part

Analogous to attribute $\texttt{edition}$ described above, the string value of this attribute can be stripped to pure number digits. Both ways, with and without letter stripping have been tried for modelling. The final decision for the best processing will be documented in chapter [Overview and Summary](./0_OverviewSummary.ipynb). Three different metrics have been tried for attribute $\texttt{part}$. Finally, metric StringCompare95 will be used.

In [49]:
attribute = 'part'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.StrCmp95()
#tedi.Jaro()
#tedi.Hamming()
#tedi.LCSStr()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [50]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

part values range [0.         0.27407407 0.28240741 0.29202279 0.30740741 0.31318681
 0.31481481 0.32407407 0.32898551 0.33333333 0.33597884 0.33838384
 0.33921569 0.34444444 0.3452381  0.35128205 0.35185185 0.35897436
 0.36111111 0.36231884 0.36666667 0.37037037 0.37179487 0.37254902
 0.37301587 0.37407407 0.375      0.37777778 0.38333333 0.38461538
 0.38568376 0.38888889 0.38927739 0.39070048 0.39215686 0.39393939
 0.3952381  0.3960114  0.39646465 0.39722222 0.39814815 0.4
 0.4031746  0.4037037  0.40604575 0.40740741 0.40842491 0.40855763
 0.41025641 0.41111111 0.41125541 0.41176471 0.41203704 0.41282051
 0.41388889 0.41449275 0.41452991 0.41507937 0.41666667 0.4178744
 0.41798942 0.41851852 0.41919192 0.42083333 0.42222222 0.42261905
 0.42390289 0.4241453  0.42564103 0.42592593 0.42777778 0.42810458
 0.42857143 0.42948718 0.43030303 0.43055556 0.43115942 0.43174603
 0.43236715 0.43333333 0.43518519 0.43557423 0.43589744 0.43627451
 0.43650794 0.43703704 0.4375     0.43813131 0.43888

In [51]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

position = uniques_len[attribute]//7

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

,duplicates,part_delta,part_x,part_y
79467,0,1.0,,
93525,0,1.0,,
25795,0,1.0,,
86274,0,1.0,,
108092,0,1.0,,
67610,0,1.0,,
94608,0,1.0,,
49384,0,1.0,,
9892,0,1.0,,
185456,0,1.0,,


0.9215686274509803 <= part_delta <= 1.0


,duplicates,part_delta,part_x,part_y
202901,0,0.0,,290 2011
254360,0,0.0,,1 12 13 15
86404,0,0.0,,23 1863
180758,0,0.0,,1
13241,0,0.0,,1
85447,0,0.0,,268 1989
188607,0,0.0,13,
223572,0,0.0,13,
176961,0,0.0,,33 33
24507,0,0.0,12,


0.0 <= part_delta <= 0.28240740740740744


,duplicates,part_delta,part_x,part_y
257261,0,0.714286,23 1862,3
214680,0,0.714286,912 912,1
257217,0,0.714286,23 1862,2
117862,0,0.714286,552 552,2
131472,0,0.714286,3,23 1863
221364,0,0.714286,4,241 319
214396,0,0.714286,912 912,1
121147,0,0.714286,2,241 319
58533,0,0.714286,2,23 1863
117739,0,0.714286,552 552,2


0.7142857142857143 <= part_delta <= 0.7146464646464646


,duplicates,part_delta,part_x,part_y
257479,0,0.717338,23 1862,23 23 1988 23
257465,0,0.717338,23 1862,23 23 1870 23
197572,0,0.717338,23 1862,23 23 1870 23
197586,0,0.717338,23 1862,23 23 1988 23
197596,0,0.717338,23 1862,23 23 1870 23
166866,0,0.716667,28 10 2013 2421 2431,2 2
197600,0,0.717338,23 1862,23 23 1899 23
257478,0,0.717338,23 1862,23 23 1946 23
257463,0,0.717338,23 1862,23 23 1906 23
257495,0,0.717338,23 1862,23 23 1889 23


0.7159544159544161 <= part_delta <= 0.7173382173382173


In this attribute, too, moving pairs of empty values to negative values will result in a clearer distinction between pairs of uniques and duplicates, as will be seen in the graphical comparison of capter [Features Discussion and Dummy Classifier Baseline](./5_FeatureDiscussionDummyBaseline.ipynb).

In [52]:
df_feature_base = dpf.mark_missing(df_feature_base, 'part', factor)

### person

As a result of chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{person}$ has been split into three specific attributes. Attribute $\texttt{person}\_{100}$ and $\texttt{person}\_{700}$ hold strongly standardised string values. For comparing pure strings, a Levenshtein metric is recommended [[Chri2012](./A_References.ipynb#chri2012)]. Unfortunately, this metric shows a very long calculation time on the data of the capstone project. Comparing the similarity values of the Levenshtein metric with the similarity values of other metrics in appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb), similarity metric StrCmp95 has been decided to use.

In [53]:
attribute = 'person'

columns_metadata_dict['similarity_metrics'][attribute+'_100'] = tedi.StrCmp95()
columns_metadata_dict['similarity_metrics'][attribute+'_700'] = tedi.StrCmp95()
#tedi.Levenshtein()

pe_values = ['_100', '_700']

for pe in pe_values :
    print('Calculating person'+pe)
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+pe,
        columns_metadata_dict['similarity_metrics'][attribute+pe],
        columns_metadata_dict)

Calculating person_100
Calculating person_700


In [54]:
pe = '_100'

uniques[attribute+pe], uniques_len[attribute+pe] = dpf.determine_similarity_values(
    df_feature_base, attribute+pe)

person_100 values range [0.         0.31944444 0.32777778 0.33095238 0.34555556 0.34768519
 0.3562963  0.35714286 0.36414141 0.36527778 0.36723647 0.37777778
 0.38511815 0.38603989 0.38779956 0.38989899 0.39246032 0.39490741
 0.40142857 0.40347222 0.40666667 0.41281046 0.4130719  0.41596491
 0.42324786 0.42388889 0.4240404  0.42486772 0.42614379 0.42620321
 0.42676768 0.42819444 0.42869281 0.42936508 0.42982456 0.42991453
 0.43208333 0.43326118 0.43415893 0.43508772 0.43555556 0.43590643
 0.43660714 0.43794872 0.43804714 0.439819   0.44040404 0.44079254
 0.44126984 0.44222222 0.44282407 0.44292929 0.44444444 0.44457516
 0.44507937 0.44571078 0.44649123 0.44656085 0.4465812  0.44662698
 0.44722222 0.44782135 0.44786325 0.44791667 0.4481352  0.44861111
 0.44972222 0.45126263 0.45131313 0.45222222 0.45291375 0.45314327
 0.45339367 0.45416667 0.45424242 0.45555556 0.45564103 0.45578704
 0.45614973 0.45784314 0.45873016 0.45952381 0.46013072 0.46055556
 0.46119281 0.46127946 0.46336996 0.46

In [55]:
position = uniques_len[attribute+pe]

dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position-2],
    uniques[attribute+pe][uniques_len[attribute+pe]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position],
    uniques[attribute+pe][uniques_len[attribute+pe]-position+2], 10)

,duplicates,person_100_delta,person_100_x,person_100_y
58541,0,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus
59114,0,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus
92375,0,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus
53299,0,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus
107526,0,1.0,,
203030,0,1.0,,
152131,0,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus
22287,0,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus
97163,0,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus
87040,0,1.0,,


0.8666666666666666 <= person_100_delta <= 1.0


,duplicates,person_100_delta,person_100_x,person_100_y
146145,0,0.0,,mortzfeldpeter
243017,0,0.0,,mozartwolfgang amadeus
41023,0,0.0,mozartwolfgang amadeus,
138767,0,0.0,rosoffmeg,
175273,0,0.0,,eusebius
79742,0,0.0,,mozartwolfgang amadeus
123303,0,0.0,,mozartwolfgang amadeus
74256,0,0.0,mozartwolfgang amadeus,
49316,0,0.0,,bührerwalter
176228,0,0.0,voltaire,


0.0 <= person_100_delta <= 0.3277777777777777


For comparing person names, like in attribute $\texttt{person}\_{245c}$, a Jaro metric will be tested [[Chri2012](./A_References.ipynb#chri2012)].

In [56]:
pe = '_245c'

columns_metadata_dict['similarity_metrics'][attribute+pe] = tedi.Jaro()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute+pe,
    columns_metadata_dict['similarity_metrics'][attribute+pe],
    columns_metadata_dict)

In [57]:
uniques[attribute+pe], uniques_len[attribute+pe] = dpf.determine_similarity_values(
    df_feature_base, attribute+pe)

person_245c values range [0.         0.24632035 0.25462963 ... 0.99583333 0.99603175 1.        ]


In [58]:
position = uniques_len[attribute+pe]

dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position-2],
    uniques[attribute+pe][uniques_len[attribute+pe]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position],
    uniques[attribute+pe][uniques_len[attribute+pe]-position+2], 10)

,duplicates,person_245c_delta,person_245c_x,person_245c_y
212707,0,1.0,,
203522,0,1.0,,
202714,0,1.0,,
204444,0,1.0,,
82642,0,1.0,wolfgang amadeus mozart,wolfgang amadeus mozart
218163,0,1.0,heidy binder... [et al.] ; [éd.:] interkantona...,heidy binder... [et al.] ; [éd.:] interkantona...
125032,0,1.0,heidy binder... [et al.] ; [éd.:] interkantona...,heidy binder... [et al.] ; [éd.:] interkantona...
182389,0,1.0,,
212685,0,1.0,,
213183,0,1.0,,


0.996031746031746 <= person_245c_delta <= 1.0


,duplicates,person_245c_delta,person_245c_x,person_245c_y
55186,0,0.0,,[heidy binder... [et al.]
135845,0,0.0,,[walter bührer]
212170,0,0.0,,[heidy binder ... et al.]
233640,0,0.0,,sigrid kessler [u.a.]
78025,0,0.0,,wolfgang amadeus mozart
62917,0,0.0,,[walter bührer]
232863,0,0.0,,wolgang amadeus mozart ; dieser opernführer wu...
200658,0,0.0,herausgegeben von birgit spinath,
14155,0,0.0,regie: volker schlöndorff ; drehbuch: volker s...,
201620,0,0.0,,sigrid kessler [u.a.]


0.0 <= person_245c_delta <= 0.25462962962962954


The similarities of all three $\texttt{person}$ attributes are affected by empty values. These will be handled the same way as the attributes above.

In [59]:
pe_values = ['_100', '_245c', '_700']

for pe in pe_values :
    df_feature_base = dpf.mark_missing(df_feature_base, 'person'+pe, factor)

### pubinit

This attribute holds publisher strings that have a similar representation as attribute $\texttt{person}$. A Jaro metric will be used.

In [60]:
attribute = 'pubinit'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaro()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [61]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

pubinit values range [0.         0.25132275 0.25303644 ... 0.98404165 0.98484848 1.        ]


In [62]:
position = uniques_len[attribute]//3

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-1],
    uniques[attribute][uniques_len[attribute]-position], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-5],
    uniques[attribute][uniques_len[attribute]-1], 10)

,duplicates,pubinit_delta,pubinit_x,pubinit_y
2900,0,0.554701,alber,"interkantonale lehrmittelzentrale ; [bern], be..."
149371,0,0.554701,erato,"interkantonale lehrmittelzentrale ; [bern], be..."
44999,0,0.554701,alber,"interkantonale lehrmittelzentrale ; [bern], be..."
199772,0,0.554701,alber,"interkantonale lehrmittelzentrale ; [bern], be..."
97756,0,0.554626,"tdk recording media europe, opernhaus",p. reclam jun.
199776,0,0.554701,alber,"interkantonale lehrmittelzentrale ; [bern], be..."
97782,0,0.554626,"tdk recording media europe, opernhaus",p. reclam jun.
2896,0,0.554701,alber,"interkantonale lehrmittelzentrale ; [bern], be..."
90664,0,0.554701,alber,"interkantonale lehrmittelzentrale ; [bern], be..."
149367,0,0.554701,erato,"interkantonale lehrmittelzentrale ; [bern], be..."


0.5546260546260546 <= pubinit_delta <= 0.5547008547008546


,duplicates,pubinit_delta,pubinit_x,pubinit_y
166368,0,1.0,,
82231,0,1.0,,
32412,0,1.0,,
15134,0,1.0,,
257429,0,1.0,,
31577,0,1.0,,
78304,0,1.0,,
197516,0,1.0,,
60706,0,1.0,,
162047,0,1.0,,


0.9777777777777779 <= pubinit_delta <= 1.0


The similarities of $\texttt{pubinit}$ are affected by empty values. These will be transformed to negative values.

In [63]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### scale

Comparing the similarity metrics of some sample value pairs of attribute $\texttt{scale}$ in appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb), a Jaccard metrics has been identified to express the best matching behaviour for purely numerical values stored in the attribute.

In [64]:
attribute = 'scale'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [65]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

scale values range [0.         0.17857143 0.21428571 0.57142857 1.        ]


In [66]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-3],
    uniques[attribute][uniques_len[attribute]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-4],
    uniques[attribute][uniques_len[attribute]-3], 10)

,duplicates,scale_delta,scale_x,scale_y
59415,0,0.0,,100000
241467,0,0.0,,100000
214785,0,0.0,,100000
124047,0,0.0,,100000
71883,0,0.0,,100000
99414,0,0.0,,100000
151931,0,0.0,,50000
182153,0,0.0,,100000
208841,0,0.0,,100000
257062,0,0.0,100000,


0.0 <= scale_delta <= 0.1785714285714286


,duplicates,scale_delta,scale_x,scale_y
96188,0,0.571429,50000,100000
69207,0,0.571429,50000,100000
182434,0,0.571429,50000,100000
227825,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
55269,0,0.571429,50000,100000
96185,0,0.571429,50000,100000
55281,0,0.571429,50000,100000
182432,0,0.571429,50000,100000
54975,0,0.571429,50000,100000
96167,0,0.571429,50000,100000


0.2142857142857143 <= scale_delta <= 0.5714285714285714


,duplicates,scale_delta,scale_x,scale_y
227830,0,0.178571,50 000 8 10 8 35 45 55 46 05,50000
227812,0,0.178571,50 000 8 10 8 35 45 55 46 05,50000
227502,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227842,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227817,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227835,0,0.178571,50 000 8 10 8 35 45 55 46 05,50000
227863,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227827,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227832,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227815,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000


0.1785714285714286 <= scale_delta <= 0.2142857142857143


Attribute $\texttt{scale}$ is filled for maps, only. Due to its sparse filling, the similarities of the attribute are affected strongly by empty values. These empty values will be marked with a special negative value.

In [67]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### ttlfull

Due to the discussion in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{ttlfull}$ has been split up into two new attributes $\texttt{ttlfull}\_\texttt{245}$ and $\texttt{ttlfull}\_\texttt{246}$ which will be compared by the same similarity metrics. A visual analysis of the values stored in the attribute, reveals a string of words, comparable to the strings in attribute $\texttt{person}\_\texttt{245c}$, above. The same similarity metric will be used for both title attributes, therefore.

In [68]:
attribute = 'ttlfull'

columns_metadata_dict['similarity_metrics'][attribute+'_245'] = tedi.Jaro()
columns_metadata_dict['similarity_metrics'][attribute+'_246'] = tedi.Jaro()

tf_values = ['_245', '_246']

for tf in tf_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+tf,
        columns_metadata_dict['similarity_metrics'][attribute+tf],
        columns_metadata_dict)

In [69]:
for tf in tf_values :
    uniques[attribute+tf], uniques_len[attribute+tf] = dpf.determine_similarity_values(
        df_feature_base, attribute+tf)

ttlfull_245 values range [0.         0.24175824 0.24358974 ... 0.99555556 0.99747475 1.        ]
ttlfull_246 values range [0.         0.27407407 0.35555556 0.36111111 0.36366612 0.36666667
 0.39444444 0.39814815 0.40238095 0.40555556 0.41140642 0.41190476
 0.41296296 0.41313131 0.4168847  0.41923077 0.41931217 0.42380952
 0.42424242 0.42484886 0.42767857 0.43003072 0.43333333 0.43758865
 0.43913043 0.43998145 0.44198966 0.44242424 0.44507736 0.44539295
 0.44722222 0.4479837  0.44848485 0.44901497 0.45       0.45011338
 0.45277778 0.45436508 0.45610717 0.45714286 0.45764895 0.45833333
 0.46094276 0.46205962 0.46527102 0.46729958 0.46851852 0.46885522
 0.46944444 0.46945938 0.47130647 0.47142857 0.47162447 0.47222222
 0.472318   0.47267613 0.47285513 0.47308219 0.47356964 0.47372609
 0.47388167 0.47417027 0.47428613 0.47431581 0.47446677 0.47462215
 0.47491364 0.47508251 0.47515152 0.47518634 0.47597067 0.47606838
 0.47613256 0.47629884 0.47642654 0.47794389 0.47829216 0.47836071
 0.4788

In [70]:
tf = '_245'
position = uniques_len[attribute+tf]

dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-position],
    uniques[attribute+tf][uniques_len[attribute+tf]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-3],
    uniques[attribute+tf][uniques_len[attribute+tf]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-4],
    uniques[attribute+tf][uniques_len[attribute+tf]-3], 10)

,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
223033,0,0.0,emma,blick in die welt
11603,0,0.0,emma,trionfi
96727,0,0.0,emma,blick in die welt
78388,0,0.0,"genéve, genff",domodossola
33853,0,0.0,arts,blick in die welt
219182,0,0.0,bildungsforschung und bildungspraxis,emma
33873,0,0.0,arts,blick in die welt
225984,0,0.0,emma,blick in die welt
89549,0,0.0,"bildungsforschung und bildungspraxis, beiheft",emma
89427,0,0.0,"bildungsforschung und bildungspraxis, beiheft",emma


0.0 <= ttlfull_245_delta <= 0.2417582417582418


,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
249,1,0.997475,sozialleistungsbetrug: sozialversicherungsbetr...,sozialleistungsbetrug: sozialversicherungsbetr...
144730,0,0.995556,"bonne chance !, cours de langue française deux...","bonne chance !, cours de langue française, deu..."
237,1,0.997475,sozialleistungsbetrug: sozialversicherungsbetr...,sozialleistungsbetrug: sozialversicherungsbetr...


0.9955555555555556 <= ttlfull_245_delta <= 0.9974747474747474


,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
1354,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
52668,0,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
63,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
49,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
128908,0,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
200068,0,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
81,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
194138,0,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
29,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
78,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."


0.9953703703703703 <= ttlfull_245_delta <= 0.9955555555555556


Attribute $\texttt{ttlfull}\_\texttt{245}$ is filled for all data rows of Swissbib's raw data as can be seen in chapter [Data Analysis](./1_DataAnalysis.ipynb). For attribute $\texttt{ttlfull}\_\texttt{245}$, the filling is below $10\%$. The data pairs with missing values will be marked with a negative value as has been done for similar cases above.

In [71]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute+'_246', factor)

### volumes

This attribute is described in chapter [Data Analysis](./1_DataAnalysis.ipynb) to hold a kind of contents that resembles the contents of attribute $\texttt{part}$. The same similarity metrics will be used for attribute $\texttt{volumes}$ as for attribute $\texttt{part}$, therefore.

In [72]:
attribute = 'volumes'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.StrCmp95()
#tedi.Jaro()
#tedi.LCSSeq()
#tedi.MongeElkan()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [73]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

volumes values range [0.         0.31318681 0.32222222 0.32905983 0.35128205 0.37301587
 0.37407407 0.38333333 0.38461538 0.3952381  0.40740741 0.41111111
 0.41666667 0.41737892 0.42857143 0.43650794 0.43703704 0.44017094
 0.44166667 0.44200244 0.44230769 0.44761905 0.4537037  0.45555556
 0.45833333 0.46296296 0.46428571 0.46581197 0.46666667 0.47008547
 0.47222222 0.47619048 0.47777778 0.48148148 0.48333333 0.48412698
 0.48611111 0.48888889 0.49007937 0.49145299 0.49206349 0.49365079
 0.4991453  0.5        0.50793651 0.51111111 0.51190476 0.52222222
 0.52380952 0.52564103 0.52777778 0.53333333 0.53653846 0.53703704
 0.53968254 0.54074074 0.54166667 0.54304029 0.54401709 0.54722222
 0.54761905 0.5491453  0.55       0.55128205 0.55555556 0.56031746
 0.56111111 0.56190476 0.56507937 0.56630037 0.56666667 0.56944444
 0.57407407 0.57478632 0.57777778 0.58148148 0.58333333 0.58730159
 0.58862434 0.58888889 0.59444444 0.5952381  0.59722222 0.5982906
 0.6        0.60119048 0.60320513 0.606837

In [74]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-3],
    uniques[attribute][uniques_len[attribute]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-4],
    uniques[attribute][uniques_len[attribute]-3], 10)

,duplicates,volumes_delta,volumes_x,volumes_y
157398,0,0.0,,1 452
220439,0,0.0,332,
61566,0,0.0,74,10
71324,0,0.0,64,1
156892,0,0.0,,1
5937,0,0.0,2,521
53491,0,0.0,1 379,244
145169,0,0.0,413,127
60513,0,0.0,71,
12000,0,0.0,64,332


0.0 <= volumes_delta <= 0.3131868131868132


,duplicates,volumes_delta,volumes_x,volumes_y
100432,0,0.916667,1 82,1 2
54818,0,0.933333,1 36,1 346
23934,0,0.916667,1 82,1 2
39561,0,0.933333,1 169,1 16
7921,0,0.916667,1 82,1 2
250334,0,0.916667,1 32,1 23
54829,0,0.933333,1 36,1 376
16434,0,0.933333,1 167,1 16
249931,0,0.933333,1 245,1 425
10888,0,0.916667,1 82,1 2


0.9166666666666666 <= volumes_delta <= 0.9333333333333332


,duplicates,volumes_delta,volumes_x,volumes_y
250458,0,0.916667,1 32,1 2
17,1,0.904762,1 169,1 0 169
247197,0,0.916667,1 45,145
8637,0,0.904762,1 0 169,1 169
250334,0,0.916667,1 32,1 23
135609,0,0.916667,191,1 91
79674,0,0.916667,1 82,1 2
107548,0,0.916667,1 82,1 2
247463,0,0.916667,1 45,145
10889,0,0.916667,1 82,1 2


0.9047619047619048 <= volumes_delta <= 0.9166666666666666


Attribute $\texttt{volumes}$ holds rows with missing data. The data pairs with missing values will be marked with a special negative value.

In [75]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

## DataFrame with Attributes and Similarity Features

The metric for each attribute of the feature DataFrame has been decided and the similarity features have been calculated. In this last step, the columns of the DataFrame are reordered in order to place the $\_\texttt{delta}$ columns close to their input origins $\_\texttt{x}$ and $\_\texttt{y}$ and some sample records are shown.

In [76]:
# Take _x, _y, and _delta columns together
fb_col_list = df_feature_base.columns.tolist()
fb_col_list.sort()
# Move target column to first place
fb_col_list.insert(0, fb_col_list.pop(fb_col_list.index('duplicates')))
# Reorder DataFrame columns
df_attribute_with_sim_feature = pd.DataFrame(df_feature_base, columns=fb_col_list)

# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(df_attribute_with_sim_feature.columns)

class_label = ['uniques', 'duplicate']

for i in class_label:
    display(df_attribute_with_sim_feature[df_attribute_with_sim_feature.duplicates==class_label.index(i)].sample(n=10))
    print(i)

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y,coordinate_N_delta,coordinate_N_x,coordinate_N_y,corporate_full_delta,corporate_full_x,corporate_full_y,doi_delta,doi_x,doi_y,edition_delta,edition_x,edition_y,exactDate_delta,exactDate_x,exactDate_y,format_postfix_delta,format_postfix_x,format_postfix_y,format_prefix_delta,format_prefix_x,format_prefix_y,isbn_delta,isbn_x,isbn_y,ismn_delta,ismn_x,ismn_y,musicid_delta,musicid_x,musicid_y,part_delta,part_x,part_y,person_100_delta,person_100_x,person_100_y,person_245c_delta,person_245c_x,person_245c_y,person_700_delta,person_700_x,person_700_y,pubinit_delta,pubinit_x,pubinit_y,scale_delta,scale_x,scale_y,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y,ttlfull_246_delta,ttlfull_246_x,ttlfull_246_y,volumes_delta,volumes_x,volumes_y
109095,0,-1.0,,,-1.0,,,-1.0,,,-1.0,,,-0.5,,3,0.250,1982aaaa,2010uuuu,0.111111,020000,030100,0.0,bk,vm,0.0,[3-499-17476-6],"[978-3-292-00266-2 (Livre du professeur, Cahie...",-1.0,,,-1.000000,,,-0.500000,7476,,-0.500000,mozartwolfgang amadeus,,0.593876,wolfgang amadeus mozart ; hrsg. von attila csa...,"[autorinnen und autoren: sigrid kessler, brigi...",0.521935,csampaiattila,"kesslersigrid, suterbrigitte, waltherrudolf",0.488600,rowohlt,interkantonale lehrmittelzentrale,-1.0,,,0.539496,"die zauberflöte, texte, materialien, kommentare","bonne chance!, 3, version ""realschule""",-1.0,,,-0.500000,282,
45645,0,-1.0,,,-1.0,,,-1.0,,,-1.0,,,-1.0,,,0.250,2007aaaa,1979uuuu,0.111111,010300,020000,0.0,vm,bk,0.0,[],[3-442-33001-7],-1.0,,,0.166667,502023,33001,-0.500000,,33001 33001,-0.500000,,mozartwolfgang amadeus,0.532738,ein film von volker schlöndorff,wolfgang amadeus mozart ; dieser opernführer w...,0.481620,"schlöndorffvolker, frischmax, shepardsam, delp...",pahlenkurt,0.593254,"kinowelt home entertainment, arthaus","w. goldmann, musikverlag b. schott's söhne",-1.0,,,0.496726,homo faber,"die zauberflöte, originalausgabe",-1.0,,,0.777778,2,252
74538,0,-1.0,,,-1.0,,,-0.5,"metropolitan operaorchestra, metropolitan oper...",,-1.0,,,-1.0,,,0.500,2000aaaa,uuuuuuuu,0.428571,010300,010000,0.0,vm,mu,1.0,[],[],-1.0,,,0.000000,073,8962,-1.000000,,,0.524621,mozartwolfgang amadeus,wolfhugo,-0.500000,w. a. mozart ; libretto: emanuel schikaneder,,-0.500000,"schikanederemanuel, hockneydavid, levinejames",,-1.000000,,,-1.0,,,0.532859,"die zauberflöte, oper in zwei aufzügen : kv 620","mörike-lieder, band 1-4",-1.0,,,0.733333,1 169,1
238229,0,-1.0,,,-1.0,,,-0.5,,schweizdirektion für entwicklung und zusammena...,-1.0,,,-1.0,,,0.500,1987aaaa,19989999,0.111111,020000,030600,0.0,bk,cr,0.0,[],[1661-1667],-1.0,,,-1.000000,,,-0.500000,2,,-1.000000,,,0.660148,sigrid kessler... [et al.] ; [éd.:] interkanto...,hrsg.: direktion für entwicklung und zusammena...,-1.000000,,,-0.500000,staatlicher lehrmittelverlag,,-1.0,,,0.622719,"bonne chance!, cours de langue française, troi...","eine welt, das deza-magazin für entwicklung un...",-0.5,,eine welt,-0.500000,135,
42898,0,-1.0,,,-1.0,,,-1.0,,,-1.0,,,-1.0,,,0.250,1993aaaa,2017uuuu,0.111111,010100,020000,0.0,mu,bk,0.0,[963-8303-08-5],[978-0-7294-1151-6],-1.0,,,-1.000000,,,-0.500000,,20,0.535227,mozartwolfgang amadeus,voltaire,0.486726,wolfgang amadeus mozart,critical edition by nicholas cronk and j.b. sh...,-1.000000,,,0.399854,könemann,voltaire foundation,-1.0,,,0.568315,"die zauberflöte, partitura","the complete works of voltaire, micromégas and...",-1.0,,,0.000000,225,467
134709,0,-1.0,,,-1.0,,,-1.0,,,-0.5,,10.5169/seals-515343,-1.0,,,0.250,1932aaaa,2014uuuu,0.428571,010200,010053,0.0,mu,bk,1.0,[],[],-1.0,,,-0.500000,10425,,-0.500000,,293 2014,0.492677,mozartwolfgang amadeus,bührerwalter,0.517929,w.a. mozart ; ... hrsg. von kurt soldan ; klav...,[walter bührer],-0.500000,"soldankurt, mozartwolfgang amadeus",,-0.500000,c.f. peters,,-1.0,,,0.544024,"die zauberflöte, oper in 2 aufzügen",blick in die welt,-1.0,,,-0.500000,1 188,
199498,0,-1.0,,,-1.0,,,-1.0,,,-1.0,,,-1.0,,,0.250,2016aaaa,1991uuuu,0.428571,020000,020053,1.0,bk,bk,0.0,[978-0-

uniques


,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y,coordinate_N_delta,coordinate_N_x,coordinate_N_y,corporate_full_delta,corporate_full_x,corporate_full_y,doi_delta,doi_x,doi_y,edition_delta,edition_x,edition_y,exactDate_delta,exactDate_x,exactDate_y,format_postfix_delta,format_postfix_x,format_postfix_y,format_prefix_delta,format_prefix_x,format_prefix_y,isbn_delta,isbn_x,isbn_y,ismn_delta,ismn_x,ismn_y,musicid_delta,musicid_x,musicid_y,part_delta,part_x,part_y,person_100_delta,person_100_x,person_100_y,person_245c_delta,person_245c_x,person_245c_y,person_700_delta,person_700_x,person_700_y,pubinit_delta,pubinit_x,pubinit_y,scale_delta,scale_x,scale_y,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y,ttlfull_246_delta,ttlfull_246_x,ttlfull_246_y,volumes_delta,volumes_x,volumes_y
198,1,-1.0,,,-1.0,,,1.0,schweizerische gesellschaft für bildungsforschung,schweizerische gesellschaft für bildungsforschung,-1.0,,,-1.0,,,1.000,19791999,19791999,1.0,030600,030600,1.0,cr,cr,1.0,[0252-9955],[0252-9955],-1.0,,,-1.0,,,-1.000000,,,-1.0,,,0.762602,[hrsg. von der schweizerischen gesellschaft fü...,hrsg.: schweizerische gesellschaft für bildung...,-1.000000,,,-0.5,"klett und balmer, universitätsverlag, universi...",,-1.0,,,0.719298,"bildungsforschung und bildungspraxis, schweize...",bildungsforschung und bildungspraxis,0.706220,"education et recherche, educazione e ricerca",educazione e ricerca,-1.000000,,
1320,1,-1.0,,,-1.0,,,-1.0,,,-1.0,,,-1.0,,,1.000,20151475,20151475,1.0,020000,020000,1.0,bk,bk,1.0,"[88-7922-121-3, 978-88-7922-121-4]",[978-88-7922-121-4],-1.0,,,-1.0,,,0.777778,1 1,1,1.0,petrarcafrancesco,petrarcafrancesco,1.000000,francesco petrarca ; commento di bernardo lapini,francesco petrarca ; commento di bernardo lapini,1.000000,lapinibernardo,lapinibernardo,1.0,"adv, biblioteca cantonale di lugano","adv, biblioteca cantonale di lugano",-1.0,,,1.000000,"trionfi, riedizione accurata dell'incunabolo c...","trionfi, riedizione accurata dell'incunabolo c...",-1.000000,,,1.000000,2,2
789,1,-1.0,,,-1.0,,,-1.0,,,-1.0,,,-1.0,,,0.750,20091990,2009uuuu,1.0,010300,010300,1.0,vm,vm,1.0,[],[],-1.0,,,1.0,502430,502430,-1.000000,,,-1.0,,,0.772755,ein film von volker schlöndorff ; nach dem rom...,regie: volker schlöndorff ; nach dem roman von...,0.836905,"schlöndorffvolker, wurlitzerrudolph, frischmax...","schlöndorffvolker, frischmax",-0.5,,kinowelt home entertainment,-1.0,,,1.000000,homo faber,homo faber,-1.000000,,,1.000000,1 109,1 109
1382,1,-1.0,,,-1.0,,,1.0,schweizerische normen-vereinigung,schweizerische normen-vereinigung,-1.0,,,-1.0,,,0.750,2011aaaa,2011uuuu,1.0,020053,020053,1.0,bk,bk,1.0,[],[],-1.0,,,-1.0,,,-1.000000,,,-1.0,,,-1.000000,,,-1.000000,,,1.0,schweizerische normen-vereinigung (snv),schweizerische normen-vereinigung (snv),-1.0,,,0.841188,health informatics - personal health device co...,informatique de santé - communication entre di...,0.885363,medizinische informatik - kommunikation von ge...,medizinische informatik - kommunikation von ge...,-1.000000,,
1202,1,-1.0,,,-1.0,,,-1.0,,,-1.0,,,1.0,6,6,0.750,2013aaaa,2013uuuu,1.0,020000,020000,1.0,bk,bk,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]",-1.0,,,-1.0,,,-1.000000,,,1.0,trappehans-joachim,trappehans-joachim,1.000000,"hans-joachim trappe, hans-peter schuster","hans-joachim trappe, hans-peter schuster",-0.500000,schusterhans-peter,,-0.5,thieme,,-1.0,,,1.000000,ekg-kurs für isabel,ekg-kurs für isabel,-1.000000,,,1.000000,325,325
1070,1,-1.0,,,-1.0,,,-1.0,,,-1.0,,,-1.0,,,0.750,1987aaaa,1987uuuu,1.0,020000,020000,1.0,bk,bk,1.0,[],[],-1.0,,,-1.0,,,-1.000000,,,-1.0,,,-1.000000,,,-1.000000,,,1.0,[staatlicher lehrmittelverlag],[staatlicher lehrmittelverlag],-1.0,,,1.000000,"bonne chance !, cours de langue française prem...","bonne chance !, cours de langue française prem...",-1.000000,,,1.000000,134,134
567,1,-1.0,,,-1.0,,,-1.0,,,-1.0,,,-1.0,,,0.750,1843aaaa,1843uuuu,1.0,020000,020000,1.0,bk,bk,1.0,[],[],-1.0,,,-1.0,,,-1.000000,,,1.0,m

duplicate


## Summary

This chapter covers the central area of feature construction. The features of the feature matrix have been generated for each attribute of Swissbib's raw data, deciding on its similarity metric. With these metric values, the feature base DataFrame has been extended and a new DataFrame with the attribute values of the pairs together with their calculated similarity value have been generated. The similarity values will be the final features for training and performance testing of the models, compare [[JudACaps](./A_References.ipynb#judacaps)].

In [77]:
columns_metadata_dict['similarity_metrics']

{'coordinate_E': LCSStr({'qval': 1, 'external': True}),
 'coordinate_N': LCSStr({'qval': 1, 'external': True}),
 'corporate_full': LCSStr({'qval': 1, 'external': True}),
 'doi': Identity({'qval': 1, 'external': True}),
 'edition': Jaccard({'qval': 1, 'as_set': False, 'external': True}),
 'exactDate': Hamming({'qval': 1, 'test_func': <function Base._ident at 0x1101fac80>, 'truncate': False, 'external': True}),
 'format_prefix': Identity({'qval': 1, 'external': True}),
 'format_postfix': Jaccard({'qval': 2, 'as_set': False, 'external': True}),
 'isbn': Identity({'qval': 1, 'external': True}),
 'ismn': Identity({'qval': 1, 'external': True}),
 'musicid': LCSStr({'qval': 1, 'external': True}),
 'part': StrCmp95({'long_strings': False, 'external': True}),
 'person_100': StrCmp95({'long_strings': False, 'external': True}),
 'person_700': StrCmp95({'long_strings': False, 'external': True}),
 'person_245c': Jaro({'qval': 1, 'long_tolerance': False, 'winklerize': False, 'external': True}),
 'pu

The similarity metric decided for each attribute has been added as an additional piece of information to the columns metadata dictionary. The following table gives this summary in a structured form and lists the metric used for each attribute. Attributes with the same font color indicate similar types of values (description column) for better orientation.

| attribute     | subtype | description | similarity metric |
| ------------- |:--------|:------------|:------------------|
|<font color='red'>[coordinate](#coordinate)</font>|<font color='red'>\_E</font>|<font color='red'>Code(9)</font>|<font color='red'>LCSStr</font>|
|               |<font color='red'>\_N</font>|<font color='red'>Code(9)</font>|<font color='red'>LCSStr</font>|
|<font color='blue'>[corporate](#corporate)</font>|<font color='blue'>\_full</font>|<font color='blue'>Name</font>|<font color='blue'>LCSStr</font>|
|<font color='green'>[doi](#doi)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='orange'>[edition](#edition)</font>|         |<font color='orange'>Number</font>|<font color='orange'>Jaccard</font>|
|<font color='black'>[exactDate](#exactDate)</font>|         |<font color='black'>Date</font>|<font color='black'>Hamming</font>|
|<font color='red'>[format](#format)</font>|<font color='red'>\_prefix</font>|<font color='red'>Code(2)</font>|<font color='red'>Identity</font>|
|               |<font color='red'>\_postfix</font>|<font color='red'>Code(6)</font>|<font color='red'>Jaccard (qval=2)</font>|
|<font color='green'>[isbn](#isbn)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='green'>[ismn](#ismn)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='green'>[musicid](#musicid)</font>|         |<font color='green'>Identifier</font>|<font color='green'>LCSStr</font>|
|<font color='orange'>[part](#part)</font>|         |<font color='orange'>Number</font>|<font color='orange'>StrCmp95</font>|
|<font color='blue'>[person](#person)</font>|<font color='blue'>\_100</font>|<font color='blue'>Name</font>|<font color='blue'>StrCmp95</font>|
|               |<font color='blue'>\_700</font>|<font color='blue'>Name</font>|<font color='blue'>StrCmp95</font>|
|               |<font color='blue'>\_245c</font>|<font color='blue'>Name</font>|<font color='blue'>Jaro</font>|
|<font color='blue'>[pubinit](#pubinit)</font>|         |<font color='blue'>Name</font>|<font color='blue'>Jaro</font>|
|<font color='orange'>[scale](#scale)</font>|         |<font color='orange'>Number</font>|<font color='orange'>Jaccard</font>|
|<font color='blue'>[ttlfull](#ttlfull)</font>|<font color='blue'>\_245</font>|<font color='blue'>String</font>|<font color='blue'>Jaro</font>|
|               |<font color='blue'>\_246</font>|<font color='blue'>String</font>|<font color='blue'>Jaro</font>|
|<font color='orange'>[volumes](#volumes)</font>|         |<font color='orange'>Number</font>|<font color='orange'>StrCmp95</font>|

### Full Feature Matrix with Target Vector Handover

To hand over the resulting DataFrame of this chapter, the DataFrame is saved into a pickle file that will be read in the next chapters [Features Discussion and Dummy Classifier Baseline](./5_FeatureDiscussionDummyBaseline.ipynb) as input.

In [78]:
# Store into compressed intermediary file
with bz2.BZ2File(os.path.join(path_goldstandard,
                       'labelled_feature_matrix_full.pkl'), 'w') as df_output_file:
    pk.dump(df_attribute_with_sim_feature, df_output_file)

The full metadata dictionary is to be persisted for handover to subsequent chapters.

In [79]:
# The target is still needed for the feature matrix
columns_metadata_dict['features'].append('duplicates')

for k in columns_metadata_dict.keys():
    print(k, '\n', columns_metadata_dict[k], '\n')

data_analysis_columns 
 ['coordinate_E', 'coordinate_N', 'corporate_full', 'doi', 'edition', 'exactDate', 'format_prefix', 'format_postfix', 'isbn', 'ismn', 'musicid', 'part', 'person_100', 'person_700', 'person_245c', 'pubinit', 'scale', 'ttlfull_245', 'ttlfull_246', 'volumes'] 

columns_to_use 
 ['duplicates', 'coordinate_E_x', 'coordinate_E_y', 'coordinate_N_x', 'coordinate_N_y', 'corporate_full_x', 'corporate_full_y', 'doi_x', 'doi_y', 'edition_x', 'edition_y', 'exactDate_x', 'exactDate_y', 'format_prefix_x', 'format_prefix_y', 'format_postfix_x', 'format_postfix_y', 'isbn_x', 'isbn_y', 'ismn_x', 'ismn_y', 'musicid_x', 'musicid_y', 'part_x', 'part_y', 'person_100_x', 'person_100_y', 'person_700_x', 'person_700_y', 'person_245c_x', 'person_245c_y', 'pubinit_x', 'pubinit_y', 'scale_x', 'scale_y', 'ttlfull_245_x', 'ttlfull_245_y', 'ttlfull_246_x', 'ttlfull_246_y', 'volumes_x', 'volumes_y'] 

similarity_metrics 
 {'coordinate_E': LCSStr({'qval': 1, 'external': True}), 'coordinate_N': L

In [80]:
# Binary intermediary metadata file
with open(os.path.join(path_goldstandard,
                       'columns_metadata.pkl'), 'wb') as dict_output_file:
    pk.dump(columns_metadata_dict, dict_output_file)